In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
im_gray=np.load('../input/l/gray_scale.npy')
im_lab=np.load('../input/ab/ab/ab1.npy')
im_gray.astype('float32')
im_lab.astype('float32')

In [ ]:
print(im_lab.shape)

In [ ]:
print(im_gray.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from skimage.color import lab2rgb

In [ ]:
imgg=np.zeros((224,224,3))
imgg[:,:,0]=im_gray[2000]
imggr=lab2rgb(imgg)
plt.imshow(imggr)

In [ ]:
img1=np.zeros((224,224,3))
img1[:,:,0]=im_gray[7000]
img1[:,:,1]=im_lab[7000,:,:,0]
img1[:,:,2]=im_lab[7000,:,:,1]

In [ ]:
img1r=lab2rgb(img1)
plt.imshow(img1r)

In [ ]:
class convnet(nn.Module):
    def __init__(self):
        super(convnet,self).__init__()
        self.conv1=nn.Conv2d(1,4,kernel_size=5,padding=2)
        self.conv2=nn.Conv2d(4,16,kernel_size=5,padding=2)
        self.conv3=nn.Conv2d(16,64,kernel_size=5,padding=2)
        self.conv4=nn.Conv2d(64,128,kernel_size=5,padding=2)
         
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),2)
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=F.max_pool2d(F.relu(self.conv3(x)),2)
        x=F.relu(self.conv4(x))
        x=x.view(-1,224,224,2)
        return x

In [ ]:
use_gpu = torch.cuda.is_available()
model=convnet()
model.cuda()
#model=model.double()
error=nn.L1Loss().cuda()
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=100,gamma=0.7)

In [ ]:
x=np.zeros((1000,224,224),dtype=np.float32)
y=np.zeros((1000,224,224,2),dtype=np.float32)
for i in range(1000):
    x[i,:,:]=im_gray[i]
    y[i,:,:,:]=im_lab[i,:,:,:]

In [ ]:
import torchvision.transforms as transforms
from torch.autograd import Variable
n_iters=10000
batch_size=10
num_epochs=100
x_tensor=torch.from_numpy(x).cuda()
y_tensor=torch.from_numpy(y).cuda()
train = torch.utils.data.TensorDataset(x_tensor,y_tensor)
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)

In [ ]:
model.train()
for i in range(num_epochs):
    print(" ")
    print(" ")
    print(" ")
    for j,(g_img,t_img) in enumerate(train_loader):
        train=g_img.view(batch_size,1,224,224).cuda()
        labels=t_img.view(batch_size,224,224,2).cuda()
        #print(train.shape)
        #train=train.float()
        #labels=labels.float()
        train.requires_grad_(True)
        labels.requires_grad_(True)
        outputs=model(train)
        #print(labels.shape)
        #print(outputs.shape)
        loss=error(outputs,labels)
        optimizer.zero_grad()
        print(loss)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        #for param in model.parameters():
        #    print(param.grad.data.sum())

In [ ]:
x_test=np.zeros((224,224),dtype=np.float32)
x_test=im_gray[3000]
x_test_tensor=torch.from_numpy(x_test).cuda()
x_test_tensor=x_test_tensor.float()
x_test_tensor=x_test_tensor.view(1,1,224,224)
y_test_tensor=model(x_test_tensor)
y_testa=y_test_tensor.detach().cpu().numpy()
y_test=np.zeros((224,224,3))
y_test[:,:,0]=im_gray[3000]
y_test[:,:,1]=y_testa[:,:,:,0]
y_test[:,:,2]=y_testa[:,:,:,1]
y_testr=lab2rgb(y_test)
plt.imshow(y_testr)